### 1. Start the Environment

In [2]:
from gym_unity.envs import UnityEnv
import numpy as np

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**_Before running the code cell below_**, change the `file_name` parameter to match the location of the Reacher Unity environment.

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [3]:
#env_name = 'unity_envs/Crawler_StaticTarget_Linux/Crawler_StaticTarget_Linux.x86_64'
env_name = 'unity_envs/Crawler_StaticTarget'
env = UnityEnv(env_name,worker_id=1,use_visual=False, multiagent=True)

INFO:mlagents.envs:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of Training Brains : 1
        Reset Parameters :
		
Unity brain name: CrawlerStaticLearning
        Number of Visual Observations (per agent): 0
        Vector Observation space size (per agent): 129
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): [20]
        Vector Action descriptions: , , , , , , , , , , , , , , , , , , , 
INFO:gym_unity:12 agents within environment.


### 2. Examine the State and Action Spaces

* Set-up: A creature with 4 arms and 4 forearms.
* Goal: The agents must move its body toward the goal direction without falling.
* CrawlerStaticTarget - Goal direction is always forward.
* CrawlerDynamicTarget- Goal direction is randomized.
* Agents: The environment contains 3 agent linked to a single Brain.
* Agent Reward Function (independent):
* +0.03 times body velocity in the goal direction.
* +0.01 times body direction alignment with goal direction.
* Brains: One Brain with the following observation/action space.
* Vector Observation space: 117 variables corresponding to position, rotation, velocity, and angular velocities of each limb plus the acceleration and angular acceleration of the body.
* Vector Action space: (Continuous) Size of 20, corresponding to target rotations for joints.
* Visual Observations: None.
* Reset Parameters: None
* Benchmark Mean Reward for CrawlerStaticTarget: 2000
* Benchmark Mean Reward for CrawlerDynamicTarget: 400

Lets print some information about the environment.

In [4]:
# number of agents
num_agents = env.number_agents
print('Number of agents:', num_agents)

# size of each action
action_size = env.action_space.shape[0]
print('Size of each action:', action_size)

# examine the state space 
states = env.reset()
state_size = env.observation_space.shape[0]
print('There are {} agents. Each observes a state with length: {}'.format(num_agents, state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 12
Size of each action: 20
There are 12 agents. Each observes a state with length: 129
The state for the first agent looks like: [ 9.99999225e-01  1.27272622e-03  0.00000000e+00  2.25000000e+00
  1.00000036e+00 -3.22882910e-12 -1.19209290e-07  3.22882975e-12
  1.00000000e+00 -3.22882910e-12  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6.06093168e-01 -1.42857209e-01 -6.06078804e-01  5.00000000e-01
  5.00000000e-01  0.00000000e+00  5.00000000e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.33339918e+00 -1.42857209e-01
 -1.33341408e+00  5.00000000e-01  0.00000000e+00  0.00000000e+00
  5.00000000e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -6.0609

### 3. Take Random Actions in the Environment

In [5]:
states = env.reset()                 # reset env and get the current state (for each agent)
scores = np.zeros(num_agents)        # initialize the score (for each agent)
step=0
while True:
    # select an action (for each agent)
    actions = list(2*np.random.rand(num_agents, action_size)-1)
    next_states,rewards,dones,_ = env.step(actions)    
    
    # update the score (for each agent)
    scores +=  rewards
    
    # roll over states to next time step
    states = next_states                               
    step+=1
    
    # exit loop if episode finished
    if np.any(dones):                                  
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 1.1336051852752764


### 4. Training the agent!

Now it's turn to train an agent to solve the environment!  When training the environment, we have to set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [6]:
import random
import datetime
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

#pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal

# imports for rendering outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# defining the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print ("using",device)

using cpu


### 3. Define policy network (Actor Critic style)

In [8]:
action_low = env.action_space.low
action_high = env.action_space.high

# define actor critic network
class ActorCritic(nn.Module):
    
    def __init__(self,state_size,action_size,action_high,action_low,hidden_size=32):
        super(ActorCritic, self).__init__()
        
        # action range
        self.action_high = torch.tensor(action_high).to(device)
        self.action_low = torch.tensor(action_low).to(device)
        
        self.std = nn.Parameter(torch.zeros(action_size))
        
        # common network
        self.fc1 = nn.Linear(state_size,512)
        
        # actor network
        self.fc2_actor = nn.Linear(512,256)
        self.fc3_action = nn.Linear(256,action_size)
        #self.fc3_std = nn.Linear(64,action_size)
        
        # critic network
        self.fc2_critic = nn.Linear(512,64)
        self.fc3_critic = nn.Linear(64,1)
    
    def forward(self,state):
        # common network
        x = F.relu(self.fc1(state))
        
        # actor network
        x_actor = F.relu(self.fc2_actor(x))
        action_mean = F.sigmoid(self.fc3_action(x_actor))
        ## rescale action mean
        action_mean_ = (self.action_high-self.action_low)*action_mean + self.action_low
        #action_std = F.sigmoid(self.fc3_std(x_actor))
        
        # critic network
        x_critic = F.relu(self.fc2_critic(x))
        v = self.fc3_critic(x_critic)
        return action_mean_,v
    
    def act(self,state,action=None):
        # converting state from numpy array to pytorch tensor on the "device"
        state = torch.from_numpy(state).float().to(device)
        action_mean,v = self.forward(state)
        prob_dist = Normal(action_mean,F.softplus(self.std))
        if action is None:
            action = prob_dist.sample()
        log_prob = prob_dist.log_prob(action).sum(-1).unsqueeze(-1)
        entropy = prob_dist.entropy().sum(-1).unsqueeze(-1)
        return {'a': action,
                'log_pi_a': log_prob,
                'ent': entropy,
                'mean': action_mean,
                'v': v}

### 4. Storage class

In [9]:
class Storage:
    def __init__(self, size, keys=None):
        if keys is None:
            keys = []
        keys = keys + ['s', 'a', 'r', 'm',
                       'v', 'q', 'pi', 'log_pi', 'ent',
                       'adv', 'ret', 'q_a', 'log_pi_a',
                       'mean']
        self.keys = keys
        self.size = size
        self.reset()

    def add(self, data):
        for k, v in data.items():
            assert k in self.keys
            getattr(self, k).append(v)

    def placeholder(self):
        for k in self.keys:
            v = getattr(self, k)
            if len(v) == 0:
                setattr(self, k, [None] * self.size)

    def reset(self):
        for key in self.keys:
            setattr(self, key, [])

    def cat(self, keys):
        data = [getattr(self, k)[:self.size] for k in keys]
        return map(lambda x: torch.cat(x, dim=0), data)

### 4. PPO agent

In [15]:
from collections import deque
from itertools import accumulate
import torch.tensor as tensor

def random_sample(indices, batch_size):
    indices = np.asarray(np.random.permutation(indices))
    batches = indices[:len(indices) // batch_size * batch_size].reshape(-1, batch_size)
    for batch in batches:
        yield batch
    r = len(indices) % batch_size
    if r:
        yield indices[-r:]
        
class Agent:
    
    def __init__(self,env,learning_rate=1e-3):
        self.env = env
        nS = state_size
        nA = action_size
        self.policy = ActorCritic(state_size=nS,hidden_size=128,action_size=nA,
                             action_low=action_low,action_high=action_high).to(device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=learning_rate, eps=1e-5)
        
        # reset the environment
        self.states = np.array(env.reset())
        
        self.episode_rewards_window = deque(maxlen=100)
        self.episode_rewards = []
        num_trajectories = 12
        self.online_rewards = np.zeros(num_trajectories)
    
        
    def train(self,max_opt_steps=1000,num_trajectories=12,rollout_length=2048,mini_batch_size=64,gamma=.99,
              target_score=-250,use_gae=False,gae_tau=0.95,PRINT_EVERY=100):
        
        for opt_step in range(max_opt_steps):
        
            storage = Storage(rollout_length)
            states = self.states
            for _ in range(rollout_length):
                
                prediction = self.policy.act(states)
                
                # send all actions to tne environment
                next_states,rewards,terminals,_ = self.env.step(list(prediction['a'].cpu().numpy()))
                
                
                next_states = np.array(next_states)
                rewards = np.array(rewards)                   
                terminals = np.array(terminals)
                
                self.online_rewards += rewards
                for i, terminal in enumerate(terminals):
                    if terminals[i]:
                        self.episode_rewards.append(self.online_rewards[i])
                        self.episode_rewards_window.append(self.online_rewards[i])
                        self.online_rewards[i] = 0
                
                storage.add(prediction)
                storage.add({'r': tensor(rewards).unsqueeze(-1).float().to(device),
                             'm': tensor(1 - terminals).unsqueeze(-1).float().to(device),
                             's': tensor(states).to(device)})
                states = next_states

            self.states = states
            prediction = self.policy.act(states)
            storage.add(prediction)
            storage.placeholder()
            
            advantages = tensor(np.zeros((num_trajectories, 1))).float().to(device)
            returns = prediction['v'].detach()
            for i in reversed(range(rollout_length)):
                returns = storage.r[i] + gamma * storage.m[i] * returns
                if not use_gae:
                    advantages = returns - storage.v[i].detach()
                else:
                    td_error = storage.r[i] + gamma * storage.m[i] * storage.v[i + 1] - storage.v[i]
                    advantages = advantages * gae_tau * gamma * storage.m[i] + td_error
                storage.adv[i] = advantages.detach()
                storage.ret[i] = returns.detach()

            states, actions, log_probs_old, returns, advantages = storage.cat(['s', 'a', 'log_pi_a', 'ret', 'adv'])
            actions = actions.detach()
            log_probs_old = log_probs_old.detach()
            advantages = (advantages - advantages.mean()) / advantages.std()
            
            ppo_ratio_clip = 0.2
            gradient_clip = 0.5
            entropy_weight = 0.0
            
            print (states.shape)
            for _ in range(10):
                sampler = random_sample(np.arange(states.size(0)), mini_batch_size)
                for batch_indices in sampler:
                    batch_indices = tensor(batch_indices).long()
                    sampled_states = states[batch_indices]
                    sampled_actions = actions[batch_indices]
                    sampled_log_probs_old = log_probs_old[batch_indices]
                    sampled_returns = returns[batch_indices]
                    sampled_advantages = advantages[batch_indices]

                    prediction = self.policy.act(sampled_states.cpu().numpy(), sampled_actions)
                    ratio = (prediction['log_pi_a'] - sampled_log_probs_old).exp()
                    obj = ratio * sampled_advantages
                    obj_clipped = ratio.clamp(1.0 - ppo_ratio_clip,
                                              1.0 + ppo_ratio_clip) * sampled_advantages
                    policy_loss = -torch.min(obj, obj_clipped).mean() - entropy_weight * prediction['ent'].mean()

                    value_loss = 0.5 * (sampled_returns - prediction['v']).pow(2).mean()

                    self.optimizer.zero_grad()
                    (policy_loss + value_loss).backward()
                    nn.utils.clip_grad_norm_(self.policy.parameters(), gradient_clip)
                    self.optimizer.step()
            
            #printing progress
            if opt_step % PRINT_EVERY == 0:
                print ("Opt step: {}\t Avg reward: {:.2f}\t std: {}".format(opt_step,np.mean(self.episode_rewards_window),
                                                                             self.policy.std))
                # save the policy
                torch.save(self.policy, 'ppo-crawler.policy')
            
            if np.mean(self.episode_rewards_window)>= target_score:
                print ("Environment solved in {} optimization steps! ... Avg reward : {:.2f}".format(opt_step-100,
                                                                                          np.mean(self.episode_rewards_window)))
                # save the policy
                torch.save(self.policy, 'ppo-crawler.policy')
                break
                
        return self.episode_rewards

### 5. Train the agent

In [16]:
# lets define and train our agent
agent = Agent(env=env,learning_rate=3e-4)

In [17]:
scores = agent.train(max_opt_steps=2000,gamma=0.99,target_score=2000,use_gae=True,PRINT_EVERY=1)

/Users/jsingh/anaconda3/envs/cv3/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0.04221916198730469 0.005315065383911133
0.0005810260772705078 0.04267001152038574
0.000576019287109375 0.005265951156616211
0.0005240440368652344 0.004873991012573242
0.0005481243133544922 0.005070924758911133
0.0005540847778320312 0.005005836486816406
0.0005261898040771484 0.00553584098815918
0.0005688667297363281 0.0051419734954833984
0.0006678104400634766 0.0051479339599609375
0.0005459785461425781 0.005640983581542969
0.0005588531494140625 0.005320072174072266
0.0005297660827636719 0.00495600700378418
0.0005540847778320312 0.004961967468261719
0.0005099773406982422 0.005400896072387695
0.0005338191986083984 0.004937171936035156
0.0005140304565429688 0.004909038543701172
0.0005230903625488281 0.004865884780883789
0.0005021095275878906 0.005447864532470703
0.0006439685821533203 0.00485992431640625
0.0005171298980712891 0.004783153533935547
0.0005240440368652344 0.0057108402252197266
0.0005609989166259766 0.005006074905395508
0.0005369186401367188 0.004894256591796875
0.0005350112915

0.0007662773132324219 0.005871772766113281
0.0006091594696044922 0.005423784255981445
0.0005590915679931641 0.005164146423339844
0.0005877017974853516 0.005052089691162109
0.0006439685821533203 0.006190061569213867
0.0005960464477539062 0.005350828170776367
0.0005650520324707031 0.005330801010131836
0.0005283355712890625 0.0054819583892822266
0.0005319118499755859 0.005106925964355469
0.0005269050598144531 0.005740165710449219
0.0005869865417480469 0.005118131637573242
0.0005469322204589844 0.005360126495361328
0.0005438327789306641 0.005265951156616211
0.0005619525909423828 0.0048980712890625
0.0005450248718261719 0.005465269088745117
0.0005311965942382812 0.005265951156616211
0.0005350112915039062 0.0050008296966552734
0.0005261898040771484 0.005013942718505859
0.0005419254302978516 0.005571126937866211
0.000518798828125 0.005101203918457031
0.0005490779876708984 0.005274772644042969
0.0005288124084472656 0.005568027496337891
0.0005481243133544922 0.004965066909790039
0.0005481243133

0.0008709430694580078 0.005796194076538086
0.0006630420684814453 0.005335092544555664
0.0005817413330078125 0.0053021907806396484
0.0005860328674316406 0.006819963455200195
0.0005819797515869141 0.00558924674987793
0.0006041526794433594 0.006064891815185547
0.0005991458892822266 0.005201816558837891
0.0005478858947753906 0.005633115768432617
0.0007231235504150391 0.005352973937988281
0.0005733966827392578 0.005175590515136719
0.0005750656127929688 0.005556821823120117
0.0007040500640869141 0.0052759647369384766
0.0005819797515869141 0.005294084548950195
0.0005660057067871094 0.004806995391845703
0.000537872314453125 0.0057370662689208984
0.0005640983581542969 0.005151033401489258
0.0005490779876708984 0.0050048828125
0.0005528926849365234 0.005022287368774414
0.0005679130554199219 0.005415201187133789
0.0005819797515869141 0.005469083786010742
0.0005757808685302734 0.005043983459472656
0.0005738735198974609 0.005166053771972656
0.0006139278411865234 0.004753828048706055
0.0005588531494

0.0005981922149658203 0.0053670406341552734
0.0010080337524414062 0.005834817886352539
0.0007791519165039062 0.0055048465728759766
0.0006840229034423828 0.005463123321533203
0.0005810260772705078 0.005625009536743164
0.0006029605865478516 0.005548954010009766
0.0005681514739990234 0.00516200065612793
0.0006592273712158203 0.005217075347900391
0.0006260871887207031 0.0051839351654052734
0.000560760498046875 0.005588054656982422
0.0005476474761962891 0.005274295806884766
0.00061798095703125 0.004805088043212891
0.0005419254302978516 0.005496025085449219
0.0005910396575927734 0.0053369998931884766
0.0005669593811035156 0.005166053771972656
0.0006690025329589844 0.005019187927246094
0.0005562305450439453 0.005484819412231445
0.0005478858947753906 0.005071163177490234
0.0005481243133544922 0.005157947540283203
0.0006520748138427734 0.00595402717590332
0.0006499290466308594 0.0051271915435791016
0.0005509853363037109 0.0050961971282958984
0.0005431175231933594 0.005112886428833008
0.00054812

...

0.0005140304565429688 0.0064089298248291016
0.0005962848663330078 0.00662994384765625
0.0006530284881591797 0.006586313247680664
0.0005688667297363281 0.006555080413818359
0.0005390644073486328 0.006224870681762695
0.0006468296051025391 0.005609989166259766
0.0006580352783203125 0.00568389892578125
0.0005970001220703125 0.006368875503540039
0.0005788803100585938 0.005971193313598633
0.0005340576171875 0.00551605224609375
0.0006690025329589844 0.006112098693847656
0.0006349086761474609 0.006346940994262695
0.0004999637603759766 0.0058231353759765625
0.0006420612335205078 0.006520986557006836
0.0005402565002441406 0.006222724914550781
0.0004990100860595703 0.005934953689575195
0.00048804283142089844 0.005833864212036133
0.0006330013275146484 0.005882978439331055
0.0004858970642089844 0.006094217300415039
0.0005390644073486328 0.0059888362884521484
0.0005362033843994141 0.005755901336669922
0.0005180835723876953 0.005945920944213867
0.0005049705505371094 0.005233049392700195
0.00054407119

0.0006198883056640625 0.006123065948486328
0.0009100437164306641 0.006564140319824219
0.0006520748138427734 0.006000995635986328
0.0005609989166259766 0.0060138702392578125
0.0006709098815917969 0.0068738460540771484
0.0005578994750976562 0.0063610076904296875
0.0005757808685302734 0.005825042724609375
0.0007801055908203125 0.005922079086303711
0.0006229877471923828 0.006115913391113281
0.0005502700805664062 0.0059278011322021484
0.0006699562072753906 0.005899906158447266
0.0006189346313476562 0.005639076232910156
0.000530242919921875 0.006274700164794922
0.0005948543548583984 0.005703926086425781
0.0005559921264648438 0.0056629180908203125
0.0005509853363037109 0.006170034408569336
0.0005409717559814453 0.005638837814331055
0.0005407333374023438 0.0056650638580322266
0.0005490779876708984 0.0054819583892822266
0.0005688667297363281 0.00628209114074707
0.0007288455963134766 0.00569915771484375
0.0005800724029541016 0.006361722946166992
0.0005500316619873047 0.005743980407714844
0.00063

0.0008041858673095703 0.006609916687011719
0.0006210803985595703 0.005960941314697266
0.0005590915679931641 0.005713939666748047
0.0006721019744873047 0.0071048736572265625
0.0007309913635253906 0.005908012390136719
0.0005631446838378906 0.0063629150390625
0.0005950927734375 0.007424831390380859
0.0006039142608642578 0.0063478946685791016
0.0006551742553710938 0.006974935531616211
0.0006699562072753906 0.0060520172119140625
0.0006849765777587891 0.006470918655395508
0.0005900859832763672 0.006237983703613281
0.0005719661712646484 0.006006002426147461
0.0005729198455810547 0.006021022796630859
0.0006339550018310547 0.006370067596435547
0.0005521774291992188 0.0063898563385009766
0.0007512569427490234 0.0058858394622802734
0.0005497932434082031 0.006431102752685547
0.0008730888366699219 0.005990028381347656
0.0006110668182373047 0.0058748722076416016
0.0005848407745361328 0.0060882568359375
0.0006530284881591797 0.0066661834716796875
0.0005829334259033203 0.006164073944091797
0.000617980

0.0005559921264648438 0.006044149398803711
0.0005819797515869141 0.006293058395385742
0.0005600452423095703 0.0067560672760009766
0.0005958080291748047 0.006088972091674805
0.0007648468017578125 0.006234169006347656
0.0005869865417480469 0.005951881408691406
0.0005519390106201172 0.006325960159301758
0.0005660057067871094 0.005950927734375
0.0006389617919921875 0.005609273910522461
0.0005331039428710938 0.005807161331176758
0.0005590915679931641 0.0063991546630859375
0.0005512237548828125 0.005827903747558594
0.0005409717559814453 0.0057830810546875
0.0005629062652587891 0.007056236267089844
0.0005650520324707031 0.005705118179321289
0.0005559921264648438 0.005693912506103516
0.0005328655242919922 0.005656242370605469
0.0005292892456054688 0.006252765655517578
0.0005512237548828125 0.005989789962768555
0.0005340576171875 0.0057430267333984375
0.0005428791046142578 0.006790876388549805
0.0005688667297363281 0.00605320930480957
0.0005400180816650391 0.005706071853637695
0.000593185424804

KeyboardInterrupt: 

### 6. Watch the smart agent

In [ ]:
# uncomment this cell to load the trained policy for Pendulum-v0
# load policy
policy =  torch.load('ppo-crawler.policy',map_location='cpu')
agent = Agent(env)
agent.policy = policy

In [ ]:
frames = []
total_rewards = np.zeros(12)

# reset the environment
states = np.array(env.reset())
value = []
r = []
for t in range(2000):
    prediction = agent.policy.act(states)
    action  = prediction['a'].cpu().numpy()
    v = prediction['v'].detach().cpu().numpy()
    #frames.append(env.render(mode='rgb_array')) 
    
    
    # send all actions to tne environment
    next_states,rewards,terminals,_ = env.step(list(action))

    next_states = np.array(next_states)
    rewards = np.array(rewards)                   
    terminals = np.array(terminals)
    
    #value.append(v.squeeze())
    #r.append(reward)
    states=next_states
    total_rewards+= rewards
    if np.any(terminals):
        for i,terminal in enumerate(terminals):
            if terminal:
                eps_reward = total_rewards[i]
                break
        break
        
print ("Total reward:",eps_reward)
#animate_frames(frames)